In [ ]:
from MySQL import MySQL_Run, Infor
import pandas as pd

Infor = {'host': '192.168.0.126',
 'user': 'root',
 'port': 3307,
 'password': '109109',
 'database': 'bus_system',
 }

MySQL_Run("show tables", Parameter = Infor)

In [ ]:
# Create

MySQL_Run("""
CREATE TABLE login_state (
    state VARCHAR(255) PRIMARY KEY,
    verifier TEXT NOT NULL,
    return_to TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

'Query executed.'

In [ ]:
# Search

Users = MySQL_Run("select * from login_state", Parameter = Infor)
Users = pd.DataFrame(Users)
Users.tail()

In [ ]:
import pandas as pd

def whoami(lineid):
    Users = MySQL_Run(f"select * from users where line_id = '{lineid}'")
    Users = pd.DataFrame(Users)
    if len(Users) == 0:
        return "None"
    else:
        return Users.iloc[0].to_dict()
    
df = whoami("Ufbcae02cac062d99d8bee745a0b4fc97")
df

In [ ]:
# MySQL_Run("""
# DROP TABLE IF EXISTS users_Test;
# """)

# MySQL_Run("""
# CREATE TABLE users_Test (
#     user_id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY,
#     line_id VARCHAR(100) UNIQUE,
#     username VARCHAR(100) NOT NULL,
#     email VARCHAR(255),
#     phone VARCHAR(20) NULL,
#     created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
#     updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
#     last_login TIMESTAMP NULL,
#     status ENUM('active','inactive') NOT NULL DEFAULT 'active',
#     reservation_status ENUM('no_reservation','pending','approved','rejected')
#         NOT NULL DEFAULT 'no_reservation',
#     preferences MEDIUMTEXT,
#     privacy_settings MEDIUMTEXT
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
# """)

MySQL_Run("""
ALTER TABLE users ADD COLUMN session_token VARCHAR(255);
""")

In [ ]:
# Descript

Users = MySQL_Run("desc users")
Users = pd.DataFrame(Users)
Users

In [1]:
from MySQL import MySQL_Run

LineID = 'Uxxxxxxxxxxx3'
UserName = 'Test01'

Results = MySQL_Run(f"""
INSERT INTO users (line_id, username)
VALUES ('{LineID}', '{UserName}');
""")

Results

'Query executed.'

In [43]:
from MySQL import MySQL_Run
import pandas as pd

Users = MySQL_Run("select * from users where line_id = 'Ufbcae02cac062d99d8bee745a0b4fc97'")
Users = pd.DataFrame(Users)
Users.tail()

,user_id,line_id,username,password,email,phone,created_at,updated_at,last_login,status,reservation_status,preferences,privacy_settings,session_token
0,124,Ufbcae02cac062d99d8bee745a0b4fc97,陳泰霖,,None,None,2025-09-14 01:12:07,2025-09-14 01:12:07,2025-09-14 01:12:07,active,no_reservation,None,None,eyJ1aWQiOiJVZmJjYWUwMmNhYzA2MmQ5OWQ4YmVlNzQ1YT...
